# Project Phoenix: RTS Strategy Dashboard

**Goal:** Develop a standalone web application to parse Startup Company save files (`sg_*.json`), integrate static game data (from the wiki), and provide Gemini-powered strategic insights. This notebook serves as the comprehensive blueprint and data dictionary for external development.

**Motto:** Solve. The. Problem. (By processing the data efficiently.)

---

## 1. Architectural Philosophy (High Cohesion / Low Coupling)

This architecture is designed for modularity and maintainability, separating static game rules from dynamic save data and AI decision-making (the Gemini Layer).

| Layer | Responsibility | Data Source |
| :--- | :--- | :--- |
| **Data Layer** | Stores static game facts (The "Known Universe"). | Parsed `game_component_database.md` JSON (or Firebase/Firestore) |
| **Input Layer** | Ingests raw game data (The "Current Reality"). | User-uploaded JSON Save File Parser |
| **Calculation Layer**| Processes reality against the known universe. | Python/JavaScript/React Hooks |
| **Strategy Layer**| Provides actionable insights and natural language explanations. | **Gemini API Call** (The AI Co-op layer) |
| **Presentation Layer**| Displays critical metrics clearly and visually. | React (Single Page App) / Tailwind CSS |

## 2. Key Save File Data Structure Paths (Input Layer)

The core functionality relies on accurately extracting the following values from the user-provided `sg_*.json` save file. These paths should be robustly handled by the JSON parser.

| Metric | JSON Path Snippet | Type | Example Use in App |
| :--- | :--- | :--- | :--- |
| **Cash-on-Hand** | `.balance` | Number | **Runway Calculation** (Cash / Burn Rate) |
| **Game Time** | `.date`, `.started` | String (Timestamp) | **Time Tracking** and historical analysis |
| **Total Users** | `.progress.products[0].users.total` | Number | **Growth Tracking** (Registered Users) |
| **Satisfaction** | `.progress.products[0].users.satisfaction` | Number (0-100) | **CRITICAL ALERT** (The 50% warning) |
| **Feature Quality** | `.featureInstances[i].quality.current` | Number | **Q/E Gap Analysis** |
| **Feature Efficiency** | `.featureInstances[i].efficiency.current` | Number | **Q/E Gap Analysis** |
| **Current Inventory** | `.inventory.[ComponentName]` | Object (Counts) | **Production Shortfall Calculation** |
| **Employee Salaries**| `.office.workstations[i].employee.salary` | Number | **Monthly Burn Rate** calculation |
| **Transactions** | `.transactions` | Array of Objects | **Financial Audit Log** (for display) |
| **Total Employees**| `len(.office.workstations)` | Number | **WIP Utilization** calculation |

### Code Snippet: Parsing a Key Feature Status

The application should iterate through the `products` array and their `featureInstances` to collect current metrics. This is essential for the Q/E Gap Analysis.

In [ ]:
// Example JavaScript/Python logic to extract feature status
function get_feature_status(save_data) {
    const features = save_data.featureInstances.map(f => ({
        name: f.featureName,
        current_quality: f.quality.current,
        max_quality: f.quality.max,
        current_efficiency: f.efficiency.current,
        max_efficiency: f.efficiency.max,
        required_module: f.requirements // Use this to check upgrade needs
    }));
    return features;
}

// In Python (using a pseudo-code JSON library)
# feature_data = save_data['featureInstances']

## 3. The Strategy Layer (Gemini API Integration)

The Gemini API will be utilized to provide the **actionable, natural language strategy** based on the calculated metrics, fulfilling the 'AI Co-op' persona.

### Prompt Structure for Strategic Recommendation (Prompt 1)

**System Instruction (Persona):**
```
Act as a world-class strategic Product Owner for an ambitious startup. Your goal is to maximize efficiency (low coupling, high cohesion) and reduce financial risk. Provide a concise, multi-step action plan for the next 5 in-game days.
```

**User Query (Inputs):**
```
The current game state is:
1. CRITICAL: User Satisfaction is only [50%] (Target > 75%).
2. RISK: Monthly Net Burn is [-$74,300]. Runway is [2.19 Months].
3. TEAM: We have [9] employees (high-speed Devs/Designers) that are [30%] utilized.
4. GOAL: Fix Low Satisfaction by upgrading Landing Page Q/E to Level 50.
5. INVENTORY SHORTFALL: We need [X] Frontend Modules and [Y] Interface Modules. Current inventory: [Z] UI Component.

Based on this data, what is the exact Production Plan for our team to fix satisfaction and minimize waste?

In [ ]:
const generateGeminiStrategy = async (metrics, componentShortfall) => {
    const systemPrompt = "Act as a world-class strategic Product Owner for an ambitious startup. Your goal is to maximize efficiency (low coupling, high cohesion) and reduce financial risk. Provide a concise, multi-step action plan for the next 5 in-game days.";
    
    const userQuery = `The current game state is: CRITICAL: User Satisfaction is only ${metrics.satisfaction}% (Target > 75%). RISK: Monthly Net Burn is ${metrics.burnRate}. Runway is ${metrics.runway}. TEAM: We have ${metrics.teamSize} employees that are ${metrics.utilization}% utilized. GOAL: Fix Low Satisfaction by upgrading Landing Page Q/E. INVENTORY SHORTFALL: We urgently need ${componentShortfall.neededModules} and ${componentShortfall.neededComponents}. What is the exact Production Plan for our team to fix satisfaction and minimize waste?`;
    
    const apiKey = ""; // Using placeholder, runtime provides key.
    const apiUrl = `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-05-20:generateContent?key=${apiKey}`;

    // ... (rest of fetch logic using provided API integration instructions)
    // Implement exponential backoff for retries.
}


## 4. Key References for Game Rules (Data Layer / Knowledge Base)

These are the essential pages on the unofficial wiki (`https://startupcompany.fandom.com/wiki/Startup_Company_Wiki`) that developers will need to translate game rules into the static JSON database.

| Reference Area | Key Wiki Page | Data to Extract |
| :--- | :--- | :--- |
| **Components/Modules** | `/Component` | Production Recipes and required employee roles. |
| **Features** | `/Feature` | Component requirements for launching and upgrading each feature (Q/E). |
| **Employees/Skills** | `/Employee_Type` | Base salary ranges, required furniture/benefits for satisfaction, and skill caps. |
| **Hosting/Servers** | `/Hosting` | Base costs and CU provided by virtual/physical servers. |

---

## 5. Next Steps for Development

1. **Implement JSON Parser:** Create the initial script to reliably read and parse the core metrics from the game's JSON save file (Section 2).
2. **Build Static Database:** Convert the data extracted from the wiki (Section 4, also reflected in `game_component_database.md`) into a structured JSON object.
3. **Wire Calculation Layer:** Implement the core calculation logic (Runway, Satisfaction Check, Q/E Gap, and Component Shortfall).
4. **Integrate Gemini:** Connect the output of the Calculation Layer to the Strategy Layer (Gemini API) using the defined prompt structure (Section 3).